# Update Production Data and Model

> author: Shizhenkun   
> email: zhenkun.shi@tib.cas.cn   
> date: 2021-12-24  

This file contains update codes for the production server. The update should be scheduled every eight weeks.

## 1. Import packages

In [1]:
import numpy as np
import pandas as pd

import os
import re
import time
import datetime
import sys
from tqdm import tqdm

import config as cfg
from functools import reduce
import matplotlib.pyplot as plt

sys.path.append("./tools/")
import funclib
import exact_ec_from_uniprot as exactec
import minitools as mtool
import embedding_esm as esmebd
# import embedding_unirep as unirep

from pandarallel import pandarallel 
pandarallel.initialize() 
import benchmark_train as btrain

%load_ext autoreload
%autoreload 2

INFO: Pandarallel will run on 80 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


## 2. Define Functions

In [9]:
# install axel for download dataset
def install_axel():
    isExists = !which axel
    if 'axel' in str(isExists[0]):
        return True
    else:
        !sudo apt install axel -y

# add missing '-' for ec number
def refill_ec(ec):   
    if ec == '-':
        return ec
    levelArray = ec.split('.')
    if  levelArray[3]=='':
        levelArray[3] ='-'
    ec = '.'.join(levelArray)
    return ec

def specific_ecs(ecstr):
    if '-' not in ecstr or len(ecstr)<4:
        return ecstr
    ecs = ecstr.split(',')
    if len(ecs)==1:
        return ecstr
    
    reslist=[]
    
    for ec in ecs:
        recs = ecs.copy()
        recs.remove(ec)
        ecarray = np.array([x.split('.') for x in recs])
        
        if '-' not in ec:
            reslist +=[ec]
            continue
        linearray= ec.split('.')
        if linearray[1] == '-':
            #l1 in l1s and l2 not empty
            if (linearray[0] in  ecarray[:,0]) and (len(set(ecarray[:,0]) - set({'-'}))>0):
                continue
        if linearray[2] == '-':
            # l1, l2 in l1s l2s, l3 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (len(set(ecarray[:,2]) - set({'-'}))>0):
                continue
        if linearray[3] == '-':
            # l1, l2, l3 in l1s l2s l3s, l4 not empty
            if (linearray[0] in  ecarray[:,0]) and (linearray[1] in  ecarray[:,1]) and (linearray[2] in  ecarray[:,2]) and (len(set(ecarray[:,3]) - set({'-'}))>0):
                continue
                
        reslist +=[ec]
    return ','.join(reslist)

#format ec
def format_ec(ecstr):
    ecArray= ecstr.split(',')
    ecArray=[x.strip() for x in ecArray] #strip blank
    ecArray=[refill_ec(x) for x in ecArray] #format ec to full
    ecArray = list(set(ecArray)) # remove duplicates
    
    return ','.join(ecArray)

## 3. Download latest data from unisprot

In [7]:
# download location ./tmp

! mv $cfg.DATADIR'uniprot_sprot_latest.dat.gz' $cfg.TEMPDIR$currenttime'_uniprot_sprot_latest.dat.gz'
install_axel()
! axel -n 10 https://ftp.uniprot.org/pub/databases/uniprot/current_release/knowledgebase/complete/uniprot_sprot.dat.gz -o ./data/uniprot_sprot_latest.dat.gz -q -c

## 4. Preprocessing

In [8]:
exactec.run_exact_task(infile=cfg.DATADIR+'uniprot_sprot_latest.dat.gz', outfile=cfg.DATADIR+'sprot_latest.tsv')

#加载数据并转换时间格式
sprot_latest = pd.read_csv(cfg.DATADIR+'sprot_latest.tsv', sep='\t',header=0) #读入文件
sprot_latest = mtool.convert_DF_dateTime(inputdf = sprot_latest)

sprot_latest.drop_duplicates(subset=['seq'], keep='first', inplace=True)
sprot_latest.reset_index(drop=True, inplace=True)

#sprot_latest format EC
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: format_ec(x))
sprot_latest['ec_number'] = sprot_latest.ec_number.parallel_apply(lambda x: specific_ecs(x))
sprot_latest['functionCounts'] = sprot_latest.ec_number.parallel_apply(lambda x: 0 if x=='-'  else len(x.split(',')))

# Trim Strging
with pd.option_context('mode.chained_assignment', None):
    sprot_latest.ec_number = sprot_latest.ec_number.parallel_apply(lambda x : str(x).strip()) #ec trim
    sprot_latest.seq = sprot_latest.seq.parallel_apply(lambda x : str(x).strip()) #seq trim

sprot_latest.to_feather(cfg.DATADIR + 'latest_sprot.feather')


565928it [04:38, 2032.67it/s]


finished use time 267.457 s


## 5. Caculation Features

In [10]:
train= pd.read_feather(cfg.DATADIR + 'latest_sprot.feather')
print('train size: {0}'.format(len(train)))

train size: 477917


In [11]:
! mv $cfg.DATADIR'sprot_latest_rep0.feather' $cfg.DATADIR'featureBank/sprot_latest_rep0.feather'
! mv $cfg.DATADIR'sprot_latest_rep32.feather' $cfg.DATADIR'featureBank/sprot_latest_rep32.feather'
! mv $cfg.DATADIR'sprot_latest_rep33.feather' $cfg.DATADIR'featureBank/sprot_latest_rep33.feather'
! mv $cfg.DATADIR'sprot_latest_unirep.feather' $cfg.DATADIR'featureBank/sprot_latest_unirep.feather'

# !pip install fair-esm
tr_rep0, tr_rep32, tr_rep33 = esmebd.get_rep_multi_sequence(sequences=train, model='esm1b_t33_650M_UR50S',seqthres=1022)
tr_rep0.to_feather(cfg.DATADIR + 'sprot_latest_rep0.feather')
tr_rep32.to_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
tr_rep33.to_feather(cfg.DATADIR + 'sprot_latest_rep33.feather')



mv: cannot stat '/home/shizhenkun/codebase/DMLF/data/sprot_latest_rep0.feather': No such file or directory
mv: cannot stat '/home/shizhenkun/codebase/DMLF/data/sprot_latest_rep32.feather': No such file or directory
mv: cannot stat '/home/shizhenkun/codebase/DMLF/data/sprot_latest_rep33.feather': No such file or directory
mv: cannot stat '/home/shizhenkun/codebase/DMLF/data/sprot_latest_unirep.feather': No such file or directory
Transferred model to GPU


 74%|██████████████████████████████████████████████████████████████████▉                        | 351401/477917 [4:36:51<1:24:55, 24.83it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)

 75%|████████████████████████████████████████████████████████████████████▋                      | 360810/477917 [4:43:53<1:17:42, 25.11it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



In [12]:
train_esm_latest = pd.read_feather(cfg.DATADIR + 'sprot_latest_rep32.feather')
train_esm_latest = train.merge(train_esm_latest, on='id', how='left')

## 6. Split X Y

In [13]:
# task 1
X_train_task1 = np.array(train_esm_latest.iloc[:,12:])
Y_train_task1 = np.array(train_esm_latest.isenzyme.astype('int')).flatten()
train_enzyme = train_esm_latest[train_esm_latest.isenzyme].reset_index(drop=True)

# task 2
X_train_task2_s = np.array(train_enzyme.iloc[:,12:])
Y_train_task2_s = train_enzyme.functionCounts.apply(lambda x : 0 if x==1 else 1).astype('int').values

train_task2M=train_enzyme[train_enzyme.functionCounts>=2].reset_index(drop=True)
X_train_task2_m = np.array(train_task2M.iloc[:,12:])
Y_train_task2_m = np.array(train_task2M.functionCounts.astype('int')-2).flatten()

#task 3
train_set_task3= funclib.split_ecdf_to_single_lines(train_enzyme.iloc[:,np.r_[0,10,5]])
train_set_task3=train_set_task3.merge(train_esm_latest.iloc[:,np.r_[0,12:1292]], on='id', how='left')

#4. 加载EC号训练数据
print('loading ec to label dict')
dict_ec_label = btrain.make_ec_label(train_label=train_set_task3['ec_number'], test_label=train_set_task3['ec_number'], file_save= cfg.FILE_EC_LABEL_DICT, force_model_update=cfg.UPDATE_MODEL)

train_set_task3['ec_label']=train_set_task3.ec_number.parallel_apply(lambda x: dict_ec_label.get(x))    
X_train_task3 = np.array(train_set_task3.iloc[:,3:])
Y_train_task3 = np.array(train_set_task3.ec_label.astype('int')).flatten()

 99%|█████████████████████████████████████████████████████████████████████████████████████████████▍| 230124/231373 [05:15<00:04, 251.08it/s]IOPub message rate exceeded.
The Jupyter server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--ServerApp.iopub_msg_rate_limit`.

Current values:
ServerApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
ServerApp.rate_limit_window=3.0 (secs)



## 7. Train Model

In [14]:
# Task 1
groundtruth, predict, predictprob, model = funclib.knnmain(X_train_task1, Y_train_task1, X_train_task1.iloc[0:10,:], Y_train_task1.iloc[0:10,:], type='binary')
joblib.dump(model, cfg.ISENZYME_MODEL)

AttributeError: 'numpy.ndarray' object has no attribute 'iloc'

In [ ]:
groundtruth, predict, predictprob, model = funclib.xgmain(X_train_task2_s, Y_train_task2_s, X_train_task2_s.iloc[0:10,:], Y_train_task2_s.iloc[0:10,:], type='binary')
joblib.dump(model, cfg.MODELDIR+'/single_multi.model')

groundtruth, predict, predictprob, model = funclib.xgmain(X_train_task2_m, Y_train_task2_m, X_train_task2_m.iloc[0:10,:], Y_train_task2_m.iloc[0:10,:], type='multi')
joblib.dump(model, cfg.MODELDIR+'/multi_many.model')

In [ ]:
cfg.FEATURE_NUM = 1280
#train
bcommon.prepare_slice_file(x_data=X_train_task3, y_data=Y_train_task3, x_file=cfg.DATADIR+'slice_train_x_esm32_latest.txt', y_file=cfg.DATADIR+'slice_train_y_esm32_latest.txt', ec_label_dict=dict_ec_label)
btrain.train_ec_slice(trainX=cfg.DATADIR+'slice_train_x_esm32_latest.txt', trainY=cfg.DATADIR+'slice_train_y_esm32_latest.txt', modelPath=cfg.MODELDIR+'/slice_esm32')

In [ ]:
latest_sprot = pd.read_feather(cfg.FILE_LATEST_SPROT_FEATHER)